In [ ]:
# @title
!pip install aiogram
!pip install nest-asyncio
!pip install openai==0.28.1
!pip install neo4j

In [ ]:
import neo4j
from neo4j import GraphDatabase

In [ ]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

from aiogram import Bot, Dispatcher, types, filters

In [ ]:
def escape_markdown(text: str) -> str:
    escape_chars = r"_*[]()~`>#+-=|{}.!"
    return "".join(f"\\{char}" if char in escape_chars else char for char in text)

In [ ]:
import openai

openai.__version__

In [ ]:
openai.api_key = #
openai.api_base = #

In [ ]:
def get_gpt(content, model="openai/gpt-4o-mini", temperature=0, max_tokens=1000): # gpt-4o gpt-3.5-turbo gpt-4o-mini
    """
    Cached call to GPT.
    """
    messages = [{"role": "user", "content": content}]

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        n=1,
        max_tokens=max_tokens,
        headers={ "X-Title": "My App" },
    )

    return response

In [ ]:
def choose_pipeline(query, **kwargs):
    # Подготовка текста для генерации ответа
    #     ВАЖНО! Если не знаешь к какой категории отнести запрос, то верни текст `Затрудняюсь ответить`.
    input_text = f"""
    Ты консультант в университете. Твоя задача помочь классифицировать запросы пользователя на следующие категории:

    ВАЖНО! Верни только название категорий.

    ```question
    {query}
    ```

    ```categories
    !НАЗВАНИЕ: Заселение и выбор общежития
    !ОПИСАНИЕ: Вопросы о процессе заселения, выборе общежития, необходимых справках и документах для проживания.

    !НАЗВАНИЕ: Документы и справки для различных процедур в вузе
    !ОПИСАНИЕ: Вопросы о необходимых справках и документах для поступления, заселения, и других формальных процедур, включая подачу через госуслуги.

    !НАЗВАНИЕ: Подписание договоров и организационные моменты общежития
    !ОПИСАНИЕ: Вопросы о подписании договоров на общежитие, процедуре их оформления и других аспектах проживания.

    !НАЗВАНИЕ: Приемная комиссия и вопросы о поступлении
    !ОПИСАНИЕ: Вопросы о времени работы приёмной комиссии, подаче заявок, формате обучения, а также организационных моментах для абитуриентов.

    !НАЗВАНИЕ: Расписание и время проведения занятий и пересдач
    !ОПИСАНИЕ: Вопросы о расписании занятий, пересдач, времени проведения мероприятий и другой организационной информации.

    !НАЗВАНИЕ: Баллы и система оценки
    !ОПИСАНИЕ: Вопросы о начислении баллов, критериях оценки, зачетах, а также просмотре результатов и сведений в личном кабинете.

    !НАЗВАНИЕ: Проблемы с доступом к платформам и загрузкой документов
    !ОПИСАНИЕ: Вопросы о технических проблемах с личным кабинетом, трудностях с загрузкой документов и других вопросах, связанных с функционалом онлайн-систем.

    !НАЗВАНИЕ: Местоположение и навигация по вузу
    !ОПИСАНИЕ: Вопросы о том, где находятся структурные подразделения вуза, аудитории, справочные центры, и прочая информация для удобной навигации.

    !НАЗВАНИЕ: Общие вопросы по организационным аспектам для студентов
    !ОПИСАНИЕ: Вопросы, касающиеся организационных моментов, таких как требования по физкультуре, порядок распределения на проекты и взаимодействие с кураторами.

    !НАЗВАНИЕ: Публикация списков поступающих и распределение мест в общежитии
    !ОПИСАНИЕ: Вопросы о списках поступающих, распределении мест и условиях заселения, а также статусах заявок.
    ```
    """

    gpt_resp = get_gpt(input_text, **kwargs)
    return gpt_resp['choices'][0]['message']['content']

In [ ]:
def choose_subtopic(query, subtopics, **kwargs):
    # Подготовка текста для генерации ответа
    input_text = f"""
    Ты консультант в университете. Твоя задача помочь найти 3 подтемы максимально похожих запрос пользователя.

    ВАЖНО! Верни список из 3 подтем через запятую из представленных в subtopics.
    ВАЖНО! Нужно вернуть только подтемы из subtopics.
    ВАЖНО! Если нет подходящей темы, то выведи темы, т.к. в них может быть нужная информация.

    ```question
    {query}
    ```

    ```subtopics
    {subtopics}
    ```
    """

    gpt_resp = get_gpt(input_text, **kwargs)
    return gpt_resp['choices'][0]['message']['content'].replace("'", '')

In [ ]:
def make_answer(query, context, **kwargs):
    # Подготовка текста для генерации ответа
    input_text = f"""
    Ты консультант в университете. Твоя задача помочь ответить на вопросы пользователя.

    ВАЖНО! Для того, чтобы ты мог отвечать на вопросы, у тебя есть информация в context.
    ВАЖНО! Если не знаешь к какой категории отнести запрос, то верни текст `Затрудняюсь ответить`.

    ```question
    {query}
    ```

    ```context
    {context}
    ```
    """

    gpt_resp = get_gpt(input_text, **kwargs)
    return gpt_resp['choices'][0]['message']['content']

In [ ]:
# get_gpt("Напиши последовательно числа от 1 до 10")['choices'][0]['message']['content']

In [ ]:
TOKEN = #

In [ ]:
return2neo4j = {
"Заселение и выбор общежития": "1. Заселение и выбор общежития",
"Документы и справки для различных процедур в вузе": "2. Документы и справки для различных процедур в вузе",
"Подписание договоров и организационные моменты общежития": "3. Подписание договоров и организационные моменты общежития",
"Приемная комиссия и вопросы о поступлении": "4. Приемная комиссия и вопросы о поступлении",
"Расписание и время проведения занятий и пересдач": "5. Расписание и время проведения занятий и пересдач",
"Баллы и система оценки": "6. Баллы и система оценки",
"Проблемы с доступом к платформам и загрузкой документов": "7. Проблемы с доступом к платформам и загрузкой документов",
"Местоположение и навигация по вузу": "8. Местоположение и навигация по вузу",
"Общие вопросы по организационным аспектам для студентов": "9. Общие вопросы по организационным аспектам для студентов",
"Публикация списков поступающих и распределение мест в общежитии": "10. Публикация списков поступающих и распределение мест в общежитии",
}

In [ ]:
URI = "neo4j+s://d6aa41d5.databases.neo4j.io"
AUTH = ("neo4j", "X2bejiqsMw_vUgj6BP39tvBPJ0PvFt5DRtEaX5FGRPE")

In [ ]:
class Neo4jConnection:
    def __init__(self, uri, auth):
        self.driver = GraphDatabase.driver(uri, auth=auth)

    def get_combined_chunks_by_subtopics(self, subtopic_names):
        with self.driver.session() as session:
            result = session.run(
                """
                UNWIND $subtopic_names AS subtopic_name
                MATCH (subtopic:Subtopic {name: subtopic_name})-[:HAS_CHUNK]->(chunk)
                RETURN collect(chunk.text) AS all_chunks
                """,
                subtopic_names=subtopic_names
            )
            record = result.single()
            chunks = record["all_chunks"] if record else []
            return "\n\n".join(chunks) if chunks else ""

    def get_subtopics(self, topic_name):
        with self.driver.session() as session:
            result = session.run(
                "MATCH (mainTopic {name: $topic_name})-[:HAS_SUBTOPIC]->(subtopic:Subtopic) "
                "RETURN collect(subtopic.name) AS subtopics",
                topic_name=topic_name
            )
            record = result.single()
            return record["subtopics"] if record else []

    def close(self):
        self.driver.close()

In [ ]:
# Создаем подключение
neo4j_conn = Neo4jConnection(URI, AUTH)

In [ ]:
neo4j_conn.get_subtopics("5. Расписание и время проведения занятий и пересдач")

In [ ]:
top3subtopics = choose_subtopic("Где я могу посмотреть расписание?",
                                neo4j_conn.get_subtopics("5. Расписание и время проведения занятий и пересдач"))
top3subtopics

In [ ]:
neo4j_conn.get_combined_chunks_by_subtopics(top3subtopics.split(','))

In [ ]:
bot = Bot(token=TOKEN)
dp = Dispatcher()

@dp.message(filters.CommandStart())
async def process_start_command(message: types.Message):
    await message.answer("Привет!\nНапиши мне что-нибудь!")

@dp.message(filters.Command('help'))
async def process_help_command(message: types.Message):
    # Отправляем сообщение обратно пользователю
    await message.answer("Напиши что-нибудь. Я отправлю это в ответ")

@dp.message()
async def echo_message(message: types.Message):
    user_query = message.text
    gpt_answer = choose_pipeline(user_query)

    # Получаем название темы из Neo4j (ваша существующая функция)
    topic_name = return2neo4j[gpt_answer]

    top3subtopics = choose_subtopic(user_query, neo4j_conn.get_subtopics(topic_name))

    # Получаем все тексты по этой подтеме
    texts = neo4j_conn.get_combined_chunks_by_subtopics(top3subtopics.split(','))

    # Объединяем тексты в одну строку
    combined_text = texts.replace('\n', '')

    final_answer = make_answer(user_query, context=combined_text)

    # Отправляем ответ
    await message.answer(
        f"📌 *Категория вопроса:* `{escape_markdown(gpt_answer)}`\n"
        f"🔍 *Название топика в БД:* `{escape_markdown(topic_name)}`\n"
        f"📂 *Подтемы:*\n`{escape_markdown(top3subtopics)}`\n"
        f"📏 *Длина контекста:* `{len(combined_text)}` символов\n"
        f"💡 *Итоговый ответ:*\n{escape_markdown(final_answer)}",
        parse_mode="MarkdownV2"
    )


async def main():
    await dp.start_polling(bot)


if __name__ == "__main__":
    asyncio.run(main())

---

In [ ]:
!pip install python-docx

In [ ]:
from docx import Document

In [ ]:
URI = #
AUTH = #
# example_name = ['bolt+ssc', 'bolt+s', 'neo4j+ssc', 'neo4j+s']
driver = GraphDatabase.driver(URI, auth=AUTH)

In [ ]:
def create_topic(tx, topic_name):
    query = "MERGE (t:Topic {name: $name}) RETURN elementId(t)"
    result = tx.run(query, name=topic_name).single()
    return result[0] if result else None


def create_subtopic(tx, subtopic_name, topic_id):
    if topic_id is None:
        print(f"Ошибка: topic_id = None для {subtopic_name}")
        return None

    query = """
    MATCH (t) WHERE elementId(t) = $topic_id
    MERGE (s:Subtopic {name: $name})
    MERGE (t)-[:HAS_SUBTOPIC]->(s)
    RETURN elementId(s)
    """
    result = tx.run(query, name=subtopic_name, topic_id=topic_id).single()
    return result[0] if result else None


def create_chunk(tx, chunk_text, subtopic_id):
    query = """
    MATCH (s) WHERE elementId(s) = $subtopic_id
    CREATE (c:Chunk {text: $text})
    CREATE (s)-[:HAS_CHUNK]->(c)
    """
    tx.run(query, text=chunk_text, subtopic_id=subtopic_id)


def parse_document(file_path):
    doc = Document(file_path)
    topics = {}
    current_topic = None
    current_subtopic = None

    for para in doc.paragraphs:
        text = para.text.strip()
        if text.isdigit() or text == "":
            continue
        if text.endswith("?"):
            current_subtopic = text
            topics[current_topic][current_subtopic] = []
        elif text.startswith(tuple(str(i) for i in range(1, 11))):
            current_topic = text
            topics[current_topic] = {}
        elif current_subtopic:
            if current_topic not in topics:
                topics[current_topic] = {}

            if current_subtopic not in topics[current_topic]:
                topics[current_topic][current_subtopic] = []

            topics[current_topic][current_subtopic].append(text)

    return topics


def load_to_neo4j(data):
    with driver.session() as session:
        for topic, subtopics in data.items():
            topic_id = session.execute_write(create_topic, topic)
            print(f"Создан topic_id: {topic_id}")  # Проверка
            if topic_id is None:
                continue  # Пропустить, если топик не создался

            for subtopic, chunks in subtopics.items():
                subtopic_id = session.execute_write(create_subtopic, subtopic, topic_id)
                print(f"Создан subtopic_id: {subtopic_id}")  # Проверка
                if subtopic_id is None:
                    continue  # Пропустить, если подтема не создалась

                for chunk in chunks:
                    session.execute_write(create_chunk, chunk, subtopic_id)

In [ ]:
if __name__ == "__main__":
    file_path = "ПД_Шатило.docx" # датасет
    parsed_data = parse_document(file_path)
    load_to_neo4j(parsed_data)
    driver.close()